# Vacation Picker

## Introduction: Using Python and the Foursquare API to pick a vacation spot

In this notebook, I will select three beach towns in New England and use the Foursquare API to fetch and analyze the venues nearby. Based on this analysis, I will create a summary description of each location and suggest who might enjoy each one the most.

---
## Data
For this project, I used location data from Foursquare, specifically the venues, their categories, and their locations, within a 1500 meter radius of each selected location.

---
## Methodology
We'll follow the same steps for each location:
1. Get the json file and convert it to a dataframe showing the venues, categories, and location
2. Summarize the nearby venues by the total number in each category
3. Create two additional dataframes, one showing eateries and another showing everything else
4. Sum up all the counts we've done - total venues, number of categories, number of eateries, number of other venues

Following that, I'll create a map for each location to visualize the location and the distribution of nearby venues. Venues will be color coded based on whether they can be described as primarily a place to get food, or another type of venue.

In [5]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

Let's pick three potential vacation spots: Block Island, an offshore island in Rhode Island; Westerly, a coastal Rhode Island town; and Cape Cod, Massachusetts. I used Google Maps to grab an address near the beach for each spot.

In [6]:
BI_address='42 Water Street, Shoreham, RI'
Westerly_address='1 Bluff Ave, Westerly, RI'
Cape_Cod_address='336 Commercial St, Provincetown, MA'

Let's get latitude and longitude for each spot.

In [7]:
address = BI_address

geolocator = Nominatim(user_agent="bi_explorer")
location = geolocator.geocode(address)
bi_latitude = location.latitude
bi_longitude = location.longitude
print('The geographical coordinates of our Block Island spot are {}, {}.'.format(bi_latitude, bi_longitude))

The geographical coordinates of our Block Island spot are 41.17262825, -71.55490556182943.


In [8]:
address = Westerly_address

geolocator = Nominatim(user_agent="wri_explorer")
location = geolocator.geocode(address)
wri_latitude = location.latitude
wri_longitude = location.longitude
print('The geographical coordinates of our Westerly spot are {}, {}.'.format(wri_latitude, wri_longitude))

The geographical coordinates of our Westerly spot are 41.3106352, -71.854421.


In [9]:
address = Cape_Cod_address

geolocator = Nominatim(user_agent="cc_explorer")
location = geolocator.geocode(address)
cc_latitude = location.latitude
cc_longitude = location.longitude
print('The geographical coordinates of our Cape Cod spot are {}, {}.'.format(cc_latitude, cc_longitude))

The geographical coordinates of our Cape Cod spot are 42.0527777, -70.1849196.


Let's put those all in the same place here:

In [10]:
print('The geographical coordinates of our Block Island spot are {}, {}.'.format(bi_latitude, bi_longitude))
print('The geographical coordinates of our Westerly spot are {}, {}.'.format(wri_latitude, wri_longitude))
print('The geographical coordinates of our Cape Cod spot are {}, {}.'.format(cc_latitude, cc_longitude))

The geographical coordinates of our Block Island spot are 41.17262825, -71.55490556182943.
The geographical coordinates of our Westerly spot are 41.3106352, -71.854421.
The geographical coordinates of our Cape Cod spot are 42.0527777, -70.1849196.


## Now let's get some information from the Foursquare API

In [1]:
CLIENT_ID = '*****'
CLIENT_SECRET = '*****'
VERSION = '*****' 
LIMIT = 100

### We'll start with Block Island
---

In [12]:
limit = 100
radius = 1500
url_bi='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
bi_latitude,
bi_longitude,
radius,
limit)

In [10]:
results=requests.get(url_bi).json()

In [11]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

First, we'll get the json file from Foursquare, clean and filter it, and put the data we want into a pandas dataframe. Then we'll view the dataframe.

In [12]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
bi_venues=nearby_venues.loc[:,filtered_columns]
bi_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
bi_venues.columns = [col.split('.')[-1] for col in bi_venues.columns]
bi_venues

,name,categories,lat,lng
0,Eli's,Seafood Restaurant,41.172964,-71.558901
1,The Ice Cream Place,Ice Cream Shop,41.172746,-71.558732
2,Ballard's Beach & Tiki Hut,Beach,41.172766,-71.554753
3,Persephone's Kitchen,Coffee Shop,41.173952,-71.560733
4,The Atlantic Inn,Hotel Bar,41.167991,-71.556781
5,Spring House Hotel,Hotel,41.168123,-71.554724
6,Koru Eco Spa,Spa,41.170147,-71.558525
7,1661 Inn,Hotel,41.169451,-71.555759
8,Mohegan Cafe & Brewery,American Restaurant,41.173618,-71.558588
9,The Beachead,Seafood Restaurant,41.175848,-71.562340


Now we have a nice list of all the venues within 1500 meters of our spot on Block Island. I chose 1500 meters, just under a mile, to give a good sense of what's within walking distance.

Let's get a broader view by summarizing the venues according to category.

In [13]:
bi_cat_count=bi_venues.categories.value_counts().to_frame()
bi_cat_count.rename(columns={'categories':'number'}, inplace=True)
bi_cat_count

,number
Seafood Restaurant,5
Beach,4
Hotel,4
Bar,3
Ice Cream Shop,3
Hotel Bar,3
American Restaurant,2
Deli / Bodega,1
Sandwich Place,1
Rental Service,1


I want to know how many places to eat are in the area, but they fall into several different categories in the Foursquare data, so I'll make a new dataframe called 'eateries' that will include every place where I could get food, excluding dessert-only venues and coffee shops.

In [14]:
bi_eateries = bi_venues[bi_venues['categories'].str.contains('Restaurant|Bar|Deli|Bake|Bakery|Pub|Food|Sandwich|Cafe|Bakery|Breakfast Spot|Bagel')].reset_index()
bi_eateries.drop(['index'],axis=1,inplace=True)
bi_eateries

,name,categories,lat,lng
0,Eli's,Seafood Restaurant,41.172964,-71.558901
1,The Atlantic Inn,Hotel Bar,41.167991,-71.556781
2,Mohegan Cafe & Brewery,American Restaurant,41.173618,-71.558588
3,The Beachead,Seafood Restaurant,41.175848,-71.562340
4,Poor People's Pub,Bar,41.173835,-71.562950
5,Ernie's Old Harbor Restaurant,Breakfast Spot,41.172253,-71.556964
6,Finns Seafood Restaurant,Seafood Restaurant,41.172410,-71.556960
7,Three Sisters,Sandwich Place,41.173181,-71.561980
8,The National Hotel Bar,Hotel Bar,41.173891,-71.559126
9,Club Soda,Bar,41.171344,-71.566891


What about places other than eateries?

In [15]:
bi_not_food=eateries = bi_venues[~bi_venues['categories'].str.contains('Restaurant|Bar|Deli|Bake|Bakery|Pub|Food|Sandwich|Cafe|Bakery|Breakfast Spot|Bagel')].reset_index()
bi_not_food.drop('index',axis=1,inplace=True)
bi_not_food

,name,categories,lat,lng
0,The Ice Cream Place,Ice Cream Shop,41.172746,-71.558732
1,Ballard's Beach & Tiki Hut,Beach,41.172766,-71.554753
2,Persephone's Kitchen,Coffee Shop,41.173952,-71.560733
3,Spring House Hotel,Hotel,41.168123,-71.554724
4,Koru Eco Spa,Spa,41.170147,-71.558525
5,1661 Inn,Hotel,41.169451,-71.555759
6,The National Hotel,Hotel,41.174101,-71.559400
7,Animal Farm,Farm,41.169115,-71.557228
8,Crescent Beach,Beach,41.179655,-71.564887
9,Blue Dory Inn,Bed & Breakfast,41.174239,-71.559961


Let's sum it up

In [16]:
bi_categories_count=bi_venues.categories.nunique()
bi_venue_count=bi_venues.name.count()
print('Number of venues in Block Island:',bi_venue_count)
print('Different types of venue in Block Island:',bi_categories_count)
print('Number of eateries in Block Island:',bi_eateries.shape[0])
print('Venues other than food places:',bi_not_food.shape[0])

Number of venues in Block Island: 41
Different types of venue in Block Island: 24
Number of eateries in Block Island: 20
Venues other than food places: 21


### Now we'll take a look at Westerly
---

In [17]:
limit = 100
radius = 1500
url_wri='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
wri_latitude,
wri_longitude,
radius,
limit)
url_wri

'https://api.foursquare.com/v2/venues/explore?&client_id=UCS1AMDHKOHRPWEAAZ2SD31VX0N1N23ZXL0KYIGZZEDP522A&client_secret=ZUTLU41WL2JGX5KVNHPC0ZNX3ZYKWAFNE1DI3ECVO3SESEZP&v=20210219&ll=41.3106352,-71.854421&radius=1500&limit=100'

In [18]:
results=requests.get(url_wri).json()

In [19]:
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
wri_venues=nearby_venues.loc[:,filtered_columns]
wri_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
wri_venues.columns = [col.split('.')[-1] for col in bi_venues.columns]
wri_venues

,name,categories,lat,lng
0,Ocean House,Hotel,41.310705,-71.853065
1,The Verandah at the Ocean House,Hotel Bar,41.310206,-71.853800
2,Watch Hill Beach,Beach,41.309821,-71.857785
3,East Beach,Beach,41.308754,-71.853610
4,St. Claire Annex,Ice Cream Shop,41.309521,-71.858079
5,Ocean House Beach Club,Beach,41.309559,-71.852174
6,Ten Sandwiches,Deli / Bodega,41.310047,-71.857754
7,Watch Hill Carousel,Theme Park,41.309366,-71.858100
8,Watch Hill Lighthouse,Lighthouse,41.304422,-71.857966
9,Napatree Point,Beach,41.310895,-71.863795


In [20]:
wri_venues.categories.value_counts().to_frame()

,categories
Beach,4
Boutique,2
American Restaurant,2
Hotel,2
Bakery,2
Harbor / Marina,2
Deli / Bodega,1
Lighthouse,1
Candy Store,1
Hobby Shop,1


In [21]:
wri_eateries = wri_venues[wri_venues['categories'].str.contains('Restaurant|Bar|Deli|Bake|Bakery')].reset_index()
wri_eateries.drop('index',axis=1,inplace=True)
wri_eateries

,name,categories,lat,lng
0,The Verandah at the Ocean House,Hotel Bar,41.310206,-71.853800
1,Ten Sandwiches,Deli / Bodega,41.310047,-71.857754
2,Olympia Tea Room,Bar,41.310243,-71.857767
3,Bay Street Deli,American Restaurant,41.309721,-71.857990
4,COAST by Jennifer Backman,New American Restaurant,41.310345,-71.853885
5,Bistro,Restaurant,41.310642,-71.854607
6,Brunas Cafe,Bakery,41.310000,-71.857740
7,Sift Bake Shop,Bakery,41.309685,-71.857733
8,St. Claire Annex,American Restaurant,41.309564,-71.858132


In [22]:
wri_not_food=wri_venues[~wri_venues['categories'].str.contains('Restaurant|Bar|Deli|Bake|Bakery')].reset_index()
wri_not_food.drop('index',axis=1,inplace=True)
wri_not_food

,name,categories,lat,lng
0,Ocean House,Hotel,41.310705,-71.853065
1,Watch Hill Beach,Beach,41.309821,-71.857785
2,East Beach,Beach,41.308754,-71.853610
3,St. Claire Annex,Ice Cream Shop,41.309521,-71.858079
4,Ocean House Beach Club,Beach,41.309559,-71.852174
5,Watch Hill Carousel,Theme Park,41.309366,-71.858100
6,Watch Hill Lighthouse,Lighthouse,41.304422,-71.857966
7,Napatree Point,Beach,41.310895,-71.863795
8,OH Spa,Spa,41.310399,-71.853413
9,Watch Hill Inn,Hotel,41.310903,-71.856922


In [23]:
wri_categories_count=wri_venues.categories.nunique()
wri_venue_count=wri_venues.name.count()
print('Number of venues in Westerly, Rhode Island:',wri_venue_count)
print('Different types of venue in Westerly:',wri_categories_count)
print('Venues other than food places:',wri_not_food.shape[0])
print('Places to eat:',wri_eateries.shape[0])

Number of venues in Westerly, Rhode Island: 30
Different types of venue in Westerly: 22
Venues other than food places: 21
Places to eat: 9


### How about Cape Cod?
---

In [24]:
limit = 100
radius = 1500
url_cc='https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
CLIENT_ID,
CLIENT_SECRET,
VERSION,
cc_latitude,
cc_longitude,
radius,
limit)
url_cc

'https://api.foursquare.com/v2/venues/explore?&client_id=UCS1AMDHKOHRPWEAAZ2SD31VX0N1N23ZXL0KYIGZZEDP522A&client_secret=ZUTLU41WL2JGX5KVNHPC0ZNX3ZYKWAFNE1DI3ECVO3SESEZP&v=20210219&ll=42.0527777,-70.1849196&radius=1500&limit=100'

In [25]:
results = requests.get(url_cc).json()

venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
wri_venues=nearby_venues.loc[:,filtered_columns]
wri_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
wri_venues.columns = [col.split('.')[-1] for col in bi_venues.columns]
wri_venues

In [26]:
venues = results['response']['groups'][0]['items']
nearby_venues=pd.json_normalize(venues)
filtered_columns = ['venue.name','venue.categories','venue.location.lat','venue.location.lng']
cc_venues=nearby_venues.loc[:,filtered_columns]
cc_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)
cc_venues.columns = [col.split('.')[-1] for col in cc_venues.columns]
cc_venues

,name,categories,lat,lng
0,Provincetown Main Street,Beach,42.051747,-70.185764
1,Far Land Provisions,Grocery Store,42.054641,-70.184882
2,Provincetown Pier,Beach,42.051079,-70.185136
3,Puzzle Me This,Hobby Shop,42.051526,-70.186403
4,141 Bradford Natural Market,Grocery Store,42.053467,-70.185827
5,Harbor Lounge,Bar,42.052936,-70.183503
6,Wired Puppy,Coffee Shop,42.053880,-70.183261
7,The Squealing Pig,American Restaurant,42.052464,-70.184997
8,Provincetown Portuguese Bakery,Bakery,42.051519,-70.186173
9,Cafe Maria,Café,42.050903,-70.186611


Let's get a birdseye view of the venues near Cape Cod

In [27]:
cc_venues['categories'].value_counts().to_frame()

,categories
Bed & Breakfast,10
American Restaurant,8
Café,6
Bar,4
Seafood Restaurant,4
Beach,4
Gay Bar,4
Coffee Shop,3
Hotel,3
Sandwich Place,3


How many of these are places to eat?

In [28]:
cc_eateries = cc_venues[cc_venues.categories.str.contains('Restaurant|Burger|Deli|Bodega|Diner|Bar|Bakery|Breakfast Spot|Pizza|Sandwich|Cafe')]
cc_eateries

,name,categories,lat,lng
5,Harbor Lounge,Bar,42.052936,-70.183503
7,The Squealing Pig,American Restaurant,42.052464,-70.184997
8,Provincetown Portuguese Bakery,Bakery,42.051519,-70.186173
11,Ross's Grill,Seafood Restaurant,42.049983,-70.187626
12,The Canteen,Sandwich Place,42.049734,-70.188366
13,Tin Pan Alley,American Restaurant,42.051001,-70.187237
14,Yolqueria,Breakfast Spot,42.054488,-70.182314
17,The Underground,Bar,42.051430,-70.186224
19,Saki,Asian Restaurant,42.051073,-70.187633
20,Cafe Edwige,Breakfast Spot,42.052351,-70.184979


In [29]:
cc_not_food = cc_venues[~cc_venues.categories.str.contains('Restaurant|Burger|Deli|Bodega|Diner|Bar|Bakery|Breakfast Spot|Pizza|Sandwich|Cafe')]

In [30]:
cc_not_food

,name,categories,lat,lng
0,Provincetown Main Street,Beach,42.051747,-70.185764
1,Far Land Provisions,Grocery Store,42.054641,-70.184882
2,Provincetown Pier,Beach,42.051079,-70.185136
3,Puzzle Me This,Hobby Shop,42.051526,-70.186403
4,141 Bradford Natural Market,Grocery Store,42.053467,-70.185827
6,Wired Puppy,Coffee Shop,42.053880,-70.183261
9,Cafe Maria,Café,42.050903,-70.186611
10,361 Coffee & Espresso Bar,Coffee Shop,42.053463,-70.183750
15,The Carpe Diem Guesthouse & Spa,Hotel,42.054296,-70.184901
16,The Penney Patch,Candy Store,42.051259,-70.186887


Let's see a quick summary of Cape Cod's amenities

In [31]:
print('Total venues near Cape Cod:',cc_venues.shape[0])
print('Different types of venue:',cc_venues.categories.nunique())
print('Places to eat:',cc_eateries.shape[0])
print('Venues excluding eateries:',cc_not_food.shape[0])

Total venues near Cape Cod: 95
Different types of venue: 50
Places to eat: 39
Venues excluding eateries: 56


## Visualization

Now, I don't know about you, but I like to get a look at some place before I decide whether to visit, so let's create some maps for our three vacation spots.

In [32]:
bi_map = folium.Map(location=[bi_latitude,bi_longitude],zoom_start=15)
for lat, lng, name, categories in zip(bi_eateries['lat'], bi_eateries['lng'], bi_eateries['name'], bi_eateries['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='orange',
                       fill=True,
                       fill_color='orange',
                       fill_opacity=.5,
                       parse_html=False).add_to(bi_map)

for lat, lng, name, categories in zip(bi_not_food['lat'], bi_not_food['lng'], bi_not_food['name'], bi_not_food['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='blue',
                       fill_opacity=.5,
                       parse_html=False).add_to(bi_map)
    
print('BLOCK ISLAND')
bi_map

BLOCK ISLAND


Interestingly, if we zoom out we'll see that Block Island is, in fact, an island! The map shows a ferry terminal near the portion of the island we've explored with our Foursquare data. Perhaps our prospective vacationer likes the idea of a ferry ride to an island vacation.

In [33]:
westerly_map = folium.Map(location=[wri_latitude,wri_longitude],zoom_start=15)
for lat, lng, name, categories in zip(wri_eateries['lat'], wri_eateries['lng'], wri_eateries['name'], wri_eateries['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='orange',
                       fill=True,
                       fill_color='orange',
                       fill_opacity=.5,
                       parse_html=False).add_to(westerly_map)

for lat, lng, name, categories in zip(wri_not_food['lat'], wri_not_food['lng'], wri_not_food['name'], wri_not_food['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='blue',
                       fill_opacity=.5,
                       parse_html=False).add_to(westerly_map)
    
print('WESTERLY, RHODE ISLAND')
westerly_map

WESTERLY, RHODE ISLAND


In [34]:
cc_map = folium.Map(location=[cc_latitude,cc_longitude],zoom_start=15)
for lat, lng, name, categories in zip(cc_eateries['lat'], cc_eateries['lng'], cc_eateries['name'], cc_eateries['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='orange',
                       fill=True,
                       fill_color='orange',
                       fill_opacity=.5,
                       parse_html=False).add_to(cc_map)

for lat, lng, name, categories in zip(cc_not_food['lat'], cc_not_food['lng'], cc_not_food['name'], cc_not_food['categories']):
    label = '{}, {}'.format(name, categories)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker([lat,lng],
                       radius=5,
                       popup=label,
                       color='blue',
                       fill=True,
                       fill_color='blue',
                       fill_opacity=.5,
                       parse_html=False).add_to(cc_map)
    
print('CAPE COD')
cc_map

CAPE COD


---
## Results

### Let's build a summary table for all three spots

In [36]:
data = {'Vacation Spot':['Block Island','Westerly, RI','Cape Cod'],
        'Total Venues':[bi_venues.shape[0],wri_venues.shape[0],cc_venues.shape[0]],
       'Eateries':[bi_eateries.shape[0],wri_eateries.shape[0],cc_eateries.shape[0]],
       'Other Venues':[bi_not_food.shape[0],wri_not_food.shape[0],cc_not_food.shape[0]],
       'Percentage Eateries':[(bi_eateries.shape[0]/bi_venues.shape[0]),wri_eateries.shape[0]/wri_venues.shape[0],cc_eateries.shape[0]/cc_venues.shape[0]]}
comp_table = pd.DataFrame(data=data).set_index('Vacation Spot')
comp_table

,Total Venues,Eateries,Other Venues,Percentage Eateries
Vacation Spot,,,,
Block Island,41,20,21,0.487805
"Westerly, RI",30,9,21,0.300000
Cape Cod,95,39,56,0.410526


---
## Discussion

We see that our three spots represent a spectrum of possibilities that may appeal to different people. If we order these three beach towns from least venues to most, we could say Westerly has the fewest offerings at 31, Block Island the next at 44, and Cape Cod by far the most at 96.
The same order, unsurprisingly, holds true if we order by number of eateries instead.
I was a bit surprised to see that Westerly has even more non-food venues than Block Island!

So, for a person who seeks a bustling atmosphere with a wide variety of things to do and places to eat, **Cape Cod** is clearly the place to go. There's sure to be something here to please almost anyone - except perhaps someone who prefers a quieter, less busy location.

For that person, **Westerly** may be the best bet, especially if they're more interested in venues that are not food joints, because Westerly has the lowest ratio of eateries to non-eateries.

And **Block Island** represents a middle ground, with a nice mix of places to eat and things to do. Besides that, it provides an opportunity to to enjoy a ferry ride and experience an island vaction.

---
## Conclusion

With the information we've discovered about the number and types of venues nearby each of our potential vacation spots, coupled with the maps for each location, I think we've enabled prospective vacationers to get a good idea of which spot would appeal most to them. Next step: packing the car!

This method could be tailored to multiple use cases, from travel websites and apps to magazine or blog writers who want an engine for generating summaries or recommendations for different destinations. We could easily use it to prioritize different types of venues for different travelers. One may want to sort destinations by the number of nearby coffee shops, or ice cream parlors, or shopping venues, or parks or beaches. 